# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [18]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [19]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [20]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [21]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the notebook root directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [6]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster(['127.0.0.1'])

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [8]:
# Create a Keyspace 
session.execute("CREATE KEYSPACE IF NOT EXISTS mykeyspace WITH REPLICATION = \
                { 'class' : 'SimpleStrategy', 'replication_factor' : 1 };")

#### Set Keyspace

In [9]:
# Set KEYSPACE to the keyspace specified above
session.set_keyspace('mykeyspace')

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




## FIRST STEP: CREATE TABLES

 - **QUERY 1:** Create **song_data** table to solve the first query. We use **PRIMARY KEY (sessionId, itemInSession)** for use this columns in where filter without dependence of another column, and to correctly insert rows for each sessionId and itemInSession combination
 - **QUERY 2:** Create **song_data_item_order** table to solve the second query. We use **PRIMARY KEY ((sessionId, userId), itemInSession)** to create **itemInSession** as a clusting column and sort the data by the cluster. The rest of PRIMARY KEY give all the combinations of **userId and sessionId** to query.
 - **QUERY 3:** Create **user_song_data** table to solve the third query. We use **PRIMARY KEY (song, userId)** for take all the combinations of **song** and **userId**.

In [10]:
header = 'Create cassandra tables:'
print(header+'\n'+'-'*len(header))

# CREATE TABLE FOR QUERY 1
create_table_song_data = """
  CREATE TABLE IF NOT EXISTS song_data ( 
  sessionId int, 
  itemInSession int, 
  artist text, 
  song text,
  length float,
  PRIMARY KEY (sessionId, itemInSession));"""

session.execute(create_table_song_data)
print('table 1 ok')

# CREATE TABLE FOR QUERY 2
create_table_song_data_item_order = """
  CREATE TABLE IF NOT EXISTS song_data_item_order ( 
  sessionId int, 
  userId int, 
  itemInSession int,
  artist text,  
  song text,
  firstName text,
  lastName text,
  PRIMARY KEY ((sessionId, userId), itemInSession));"""

session.execute(create_table_song_data_item_order)
print('table 2 ok')

# CREATE TABLE FOR QUERY 3
create_table_song_data_item_order = """
  CREATE TABLE IF NOT EXISTS user_song_data (
  song text,
  userId int,
  firstName text,
  lastName text,
  PRIMARY KEY (song, userId));"""

session.execute(create_table_song_data_item_order)
print('table 3 ok')

Create cassandra tables:
------------------------
table 1 ok
table 2 ok
table 3 ok


## SECOND STEP: INSERT DATA

 - Here we have a simple insert process of load the csv file and for each row-array insert the correct data in each table.

In [11]:
# INSERT DATA INTO TABLE song_data (QUERY 1)
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO song_data (sessionID, itemInSession, artist, song, length) "
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

print('ok')        

In [12]:
# INSERT DATA INTO TABLE song_data_item_order (QUERY 2)
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO song_data_item_order (sessionID, userId, itemInSession, artist, song, firstName, lastName) "
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), int(line[10]), int(line[3]),
                                                  line[0], line[9], line[1], line[4]))

print('ok')        

ok


In [13]:
# INSERT DATA INTO TABLE user_song_data (QUERY 3)
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO user_song_data (song, userId, firstName, lastName) "
        query = query + "VALUES (%s, %s, %s, %s)"
        session.execute(query, (line[9],int(line[10]), line[1], line[4]))
        
print('ok')

ok


## THIRD STEP: TEST THE INSERTED DATA

#### Do a SELECT to verify that the data have been inserted into each table

In [14]:
# Table array to interact into the foreach
tables = ['song_data','song_data_item_order', 'user_song_data']

for table in tables:
    query = f"SELECT * FROM {table} LIMIT 1"
    try:
        rows = session.execute(query)
    except Exception as e:
        print(e)

    for row in rows:
        print(table + '\n---------\n' + str(row) + '\n')

song_data
---------
Row(sessionid=23, iteminsession=0, artist='Regina Spektor', length=191.08526611328125, song='The Calculation (Album Version)')

song_data_item_order
---------
Row(sessionid=1049, userid=73, iteminsession=0, artist='Manowar', firstname='Jacob', lastname='Klein', song='Shell Shock')

user_song_data
---------
Row(song="Wonder What's Next", userid=49, firstname='Chloe', lastname='Cuevas')



## FOURTH STEP: EXECUTE THE QUERY'S

- **Query 1:** Because we filter all the primary key **(sessionID, itemInSession)** in the **WHERE** condition, we can pass all the other columns in this query.  

   ~~~sql 
          SELECT artist, song, length FROM song_data WHERE sessionID = 338 and itemInSession = 4 
   ~~~
   
   
  <br><br> 
- **Query 2:** To retrieve the data of no primary key columns we need to filter all the partition columns **(userId, sessionId)**, in this query we do not need to do anything for sort the result, because the **itemInSession** is a clustering column, ans cassandra sort the data by the clustering column.  

   ~~~sql 
        SELECT artist, song, firstName, lastName FROM song_data_item_order WHERE userId = 10 and sessionId = 182
   ~~~
   <br><br>
- **Query 3:** In this query, we only need to filter the partition key **song** to access the other columns. **userId** is not selected because we create this column only for prevent duplicate data.

   ~~~sql 
        SELECT firstName, lastName FROM user_song_data
        WHERE song = 'All Hands Against His Own'
   ~~~
   
   

In [15]:
## Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4

query = "SELECT artist, song, length FROM song_data WHERE sessionID = 338 and itemInSession = 4"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist, row.song, row.length)
                    

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


In [16]:
## Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182

query = "SELECT artist, song, firstName, lastName FROM song_data_item_order "
query += "WHERE userId = 10 and sessionId = 182"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist, row.song, row.firstname, row.lastname)
                    

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


In [17]:
## Query 3: Give me every user name (first and last) in my music app history who listened to 
## the song 'All Hands Against His Own'

query = "SELECT firstName, lastName FROM user_song_data "
query += "WHERE song = 'All Hands Against His Own'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.firstname, row.lastname)
                    

Jacqueline Lynch
Tegan Levine
Sara Johnson


### Drop the tables before closing out the sessions

In [78]:
session.execute('DROP TABLE IF EXISTS song_data')
session.execute('DROP TABLE IF EXISTS song_data_item_order')
session.execute('DROP TABLE IF EXISTS user_song_data')
print('ok')

ok


### Close the session and cluster connection¶

In [32]:
session.shutdown()
cluster.shutdown()